In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import plotly
plotly.tools.set_credentials_file(username='quoniammm', api_key='IF7kV6idFRdoo7LdgGRp')
import plotly.plotly as py
import plotly.graph_objs as go
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

import nltk

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

from collections import defaultdict

import keras
import keras.backend as K
from keras.layers import Dense, GlobalAveragePooling1D, Embedding
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

%matplotlib inline

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

df = pd.read_csv('train.csv')
a2c = {'EAP': 0, 'HPL' : 1, 'MWS' : 2}
y = np.array([a2c[a] for a in df.author])
y = to_categorical(y)

Using TensorFlow backend.
/home/quoniammm/anaconda3/envs/py3Tfgpu/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning:

compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6



In [4]:
df = train

def preprocess(text):
    text = text.replace("' ", " ' ")
    signs = set(',.:;"?!')
    prods = set(text) & signs
    if not prods:
        return text

    for sign in prods:
        text = text.replace(sign, ' {} '.format(sign) )
    return text

def create_docs(df, n_gram_max=2):
    def add_ngram(q, n_gram_max):
            ngrams = []
            for n in range(2, n_gram_max+1):
                for w_index in range(len(q)-n+1):
                    ngrams.append('--'.join(q[w_index:w_index+n]))
            return q + ngrams
        
    docs = []
    for doc in df.text:
        doc = preprocess(doc).split()
        docs.append(' '.join(add_ngram(doc, n_gram_max)))
    
    return docs

min_count = 2

docs = create_docs(df)
print(docs[0])
tokenizer = Tokenizer(lower=False, filters='')
tokenizer.fit_on_texts(docs)
num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])

tokenizer = Tokenizer(num_words=num_words, lower=False, filters='')
tokenizer.fit_on_texts(docs)
docs = tokenizer.texts_to_sequences(docs)
print(docs[0])
maxlen = 256

docs = pad_sequences(sequences=docs, maxlen=maxlen)

This process , however , afforded me no means of ascertaining the dimensions of my dungeon ; as I might make its circuit , and return to the point whence I set out , without being aware of the fact ; so perfectly uniform seemed the wall . This--process process--, ,--however however--, ,--afforded afforded--me me--no no--means means--of of--ascertaining ascertaining--the the--dimensions dimensions--of of--my my--dungeon dungeon--; ;--as as--I I--might might--make make--its its--circuit circuit--, ,--and and--return return--to to--the the--point point--whence whence--I I--set set--out out--, ,--without without--being being--aware aware--of of--the the--fact fact--; ;--so so--perfectly perfectly--uniform uniform--seemed seemed--the the--wall wall--.
[174, 6008, 1, 224, 1, 2481, 26, 46, 469, 3, 20045, 2, 4827, 3, 15, 10367, 14, 21, 7, 120, 282, 59, 9408, 1, 5, 482, 6, 2, 393, 4601, 7, 533, 106, 1, 206, 182, 1587, 3, 2, 506, 14, 49, 2645, 11508, 142, 2, 725, 4, 20046, 245, 273, 45016, 9409,

In [15]:
input_dim = np.max(docs) + 1
embedding_dims = 20

In [16]:
def create_model(embedding_dims=20, optimizer='adam'):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=embedding_dims))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

In [17]:
epochs = 25
x_train, x_test, y_train, y_test = train_test_split(docs, y, test_size=0.2)

model = create_model()
hist = model.fit(x_train, y_train,
                 batch_size=16,
                 validation_data=(x_test, y_test),
                 epochs=epochs,
                 callbacks=[EarlyStopping(patience=2, monitor='val_loss')])

Train on 15663 samples, validate on 3916 samples
Epoch 1/25
15663/15663 [==============================] - 23s 1ms/step - loss: 1.0650 - acc: 0.4216 - val_loss: 1.0275 - val_acc: 0.5102
Epoch 2/25
15663/15663 [==============================] - 25s 2ms/step - loss: 0.9256 - acc: 0.6060 - val_loss: 0.8659 - val_acc: 0.6849
Epoch 3/25
15663/15663 [==============================] - 23s 1ms/step - loss: 0.7345 - acc: 0.7747 - val_loss: 0.7307 - val_acc: 0.73980s - loss: 0.7359 - acc
Epoch 4/25
15663/15663 [==============================] - 23s 1ms/step - loss: 0.5814 - acc: 0.8355 - val_loss: 0.6324 - val_acc: 0.7753
Epoch 5/25
15663/15663 [==============================] - 22s 1ms/step - loss: 0.4668 - acc: 0.8745 - val_loss: 0.5595 - val_acc: 0.8110
Epoch 6/25
15663/15663 [==============================] - 22s 1ms/step - loss: 0.3789 - acc: 0.9027 - val_loss: 0.5112 - val_acc: 0.8169
Epoch 7/25
15663/15663 [==============================] - 24s 2ms/step - loss: 0.3083 - acc: 0.9261 - val_

In [18]:
docs = create_docs(df)
tokenizer = Tokenizer(lower=True, filters='')
tokenizer.fit_on_texts(docs)
num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])

tokenizer = Tokenizer(num_words=num_words, lower=True, filters='')
tokenizer.fit_on_texts(docs)
docs = tokenizer.texts_to_sequences(docs)

maxlen = 256

docs = pad_sequences(sequences=docs, maxlen=maxlen)

input_dim = np.max(docs) + 1

In [19]:
epochs = 16
x_train, x_test, y_train, y_test = train_test_split(docs, y, test_size=0.2)

model = create_model()
hist = model.fit(x_train, y_train,
                 batch_size=16,
                 validation_data=(x_test, y_test),
                 epochs=epochs,
                 callbacks=[EarlyStopping(patience=2, monitor='val_loss')])

Train on 15663 samples, validate on 3916 samples
Epoch 1/16
15663/15663 [==============================] - 22s 1ms/step - loss: 1.0696 - acc: 0.4110 - val_loss: 1.0312 - val_acc: 0.4890
Epoch 2/16
15663/15663 [==============================] - 23s 1ms/step - loss: 0.9378 - acc: 0.6033 - val_loss: 0.8593 - val_acc: 0.7071
Epoch 3/16
15663/15663 [==============================] - 23s 1ms/step - loss: 0.7377 - acc: 0.7831 - val_loss: 0.7090 - val_acc: 0.7694
Epoch 4/16
15663/15663 [==============================] - 22s 1ms/step - loss: 0.5806 - acc: 0.8405 - val_loss: 0.6086 - val_acc: 0.7937
Epoch 5/16
15663/15663 [==============================] - 22s 1ms/step - loss: 0.4666 - acc: 0.8761 - val_loss: 0.5406 - val_acc: 0.8082
Epoch 6/16
15663/15663 [==============================] - 21s 1ms/step - loss: 0.3809 - acc: 0.9031 - val_loss: 0.4959 - val_acc: 0.8292
Epoch 7/16
15663/15663 [==============================] - 23s 1ms/step - loss: 0.3131 - acc: 0.9254 - val_loss: 0.4541 - val_acc:

In [21]:
test_df = pd.read_csv('test.csv')
docs = create_docs(test_df)
docs = tokenizer.texts_to_sequences(docs)
docs = pad_sequences(sequences=docs, maxlen=maxlen)
y = model.predict_proba(docs)

result = pd.read_csv('sample_submission.csv')
for a, i in a2c.items():
    result[a] = y[:, i]

In [22]:
result.to_csv('fastText_result.csv', index=False)